In [177]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import time
import pandas as pd

In [178]:
def try_index(list, index):
    try:
        var = list[index].text
    except:
        var = ''

    return var

In [183]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="/Users/stanvanklink/OneDrive/Analytics projects/salary predictions/ds_salary_predictions/chromedriver", options=options)
    driver.set_window_size(1120, 1000)

    url = "https://www.glassdoor.nl/Vacature/" + keyword + "-vacatures-SRCH_KO0,12.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=&typedLocation=&context=Jobs&dropdown=0"
    driver.get(url)
    jobs = []

    # wait for the cookies pop-up to show and close it
    time.sleep(5)
    try:
        driver.find_element(
            by="xpath", value='.//button[@id="onetrust-accept-btn-handler"]').click()
    except:
        pass

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Going through each job in this page
        job_buttons = driver.find_elements(by="xpath", value='.//article[@id="MainCol"]//ul/li[@data-adv-type="GENERAL"]')
        for job_button in job_buttons:
        
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            stale = True
            while(stale):
                try:
                    job_button.click()
                    stale = False
                except StaleElementReferenceException:
                    stale = True

            time.sleep(1)

            try:
                driver.find_element(by="xpath", value='.//span[@class="SVGInline modal_closeIcon"]').click()
            except NoSuchElementException:
                pass

            collected_successfully = False
            
            while not collected_successfully:
                
                try:
                    company_name = driver.find_element(by="xpath", value='.//div[@class="css-87uc0g e1tk4kwz1"]').text
                except NoSuchElementException:
                    company_name = ''
                
                try:    
                    location = driver.find_element(by="xpath", value='.//div[@class="css-56kyx5 e1tk4kwz5"]').text
                except NoSuchElementException:
                    location = ''
                
                try:
                    job_title = driver.find_element(by="xpath", value='.//div[@class="css-1vg6q84 e1tk4kwz4"]').text
                except NoSuchElementException:
                    job_title = ''
                
                try:
                    salary = driver.find_element(by="xpath", value='.//div[@class="css-1bluz6i e2u4hf13"]').text
                except NoSuchElementException:
                    salary = ''
                
                try:
                    salary_range_list = driver.find_elements(by="xpath", value='.//span[@class="css-16uanij e1wijj242"]')
                    salary_range = salary_range_list[-1].text
                except NoSuchElementException:
                    salary_range = ''

                try:
                    company_overview_list = driver.find_elements(by="xpath", value='.//span[@class="css-i9gxme e1pvx6aw2"]')
                    
                    size = try_index(company_overview_list, 0)
                    founded = try_index(company_overview_list, 1)
                    type_of_ownership = try_index(company_overview_list, 2)
                    industry = try_index(company_overview_list, 3)
                    sector = try_index(company_overview_list, 4)
                    revenue = try_index(company_overview_list, 5)

                except NoSuchElementException:
                    size = ''
                    founded = ''
                    type_of_ownership = ''
                    industry = ''
                    sector = ''
                    revenue = ''

                collected_successfully = True

            jobs.append({
                "company_name" : company_name,   #.split("\n", 1)[0],
                "location" : location,
                "job_title" : job_title,
                "salary" : salary,
                "salaray_range": salary_range,
                "size" : size,
                "founded" : founded,
                "type_of_ownership" : type_of_ownership,
                "industry" : industry,
                "sector" : sector,
                "revenue" : revenue
                })

         #Clicking on the "next page" button
        try:
            driver.find_element(by="xpath", value='.//button[@class="nextButton css-1hq9k8 e13qs2071"]').click()
            time.sleep(10)
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.
    


In [184]:
#This line will open a new chrome window and start the scraping.
df = get_jobs("data-science", 100, False)
df

# filter op alleen goede regels?

/var/folders/jg/1ck3yxvn1sn72cwsg5c615740000gn/T/ipykernel_7325/1173640601.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="/Users/stanvanklink/OneDrive/Analytics projects/salary predictions/ds_salary_predictions/chromedriver", options=options)


Progress: 0/100
Progress: 1/100
Progress: 2/100
Progress: 3/100
Progress: 4/100
Progress: 5/100
Progress: 6/100
Progress: 7/100
Progress: 8/100
Progress: 9/100
Progress: 10/100
Progress: 11/100
Progress: 12/100
Progress: 13/100
Progress: 14/100
Progress: 15/100
Progress: 16/100
Progress: 17/100
Progress: 18/100
Progress: 19/100
Progress: 20/100
Progress: 21/100
Progress: 22/100
Progress: 23/100
Progress: 24/100
Progress: 25/100
Progress: 26/100
Progress: 27/100
Progress: 28/100
Progress: 29/100
Progress: 30/100
Progress: 31/100
Progress: 32/100
Progress: 33/100
Progress: 34/100
Progress: 35/100
Progress: 36/100
Progress: 37/100
Progress: 38/100
Progress: 39/100
Progress: 40/100
Progress: 41/100
Progress: 42/100
Progress: 43/100
Progress: 44/100
Progress: 45/100
Progress: 46/100
Progress: 47/100
Progress: 48/100
Progress: 49/100
Progress: 50/100
Progress: 51/100
Progress: 52/100
Progress: 53/100
Progress: 54/100
Progress: 55/100
Progress: 56/100
Progress: 57/100
Progress: 58/100
Progres

,company_name,location,job_title,salary,salaray_range,size,founded,type_of_ownership,industry,sector,revenue
0,a.s.r.,Utrecht,Data Scientist,€ 4.050 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 5K,1001 tot 5000 werknemers,1720,Privébedrijf,Verzekeringsmaatschappijen,Verzekeringen,Minder dan 1 miljoen USD
1,Ordina,Nieuwegein,Data Scientist,,€ 1K (Schatting werkgever),1001 tot 5000 werknemers,1973,Beursgenoteerd bedrijf,IT-ondersteuning,IT,100-500 miljoen USD
2,Genmab,Utrecht,Discovery Data Scientist,,€ 1K (Schatting werkgever),501 tot 1000 werknemers,1999,Beursgenoteerd bedrijf,Biotechnologie en farmacie,Farmaceutische industrie en biotechnologie,Onbekend/niet van toepassing
3,Centric,IJsselstein,Data Scientist,€ 4.250 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 6K,1001 tot 5000 werknemers,1992,Privébedrijf,IT-ondersteuning,IT,100-500 miljoen USD
4,a.s.r.,Utrecht,Datawarehouse Ontwikkelaar en Data Analist Ver...,€ 5.550 /mnd (schatting),Salaris opgegeven door werknemer:€ 5K - € 7K,1001 tot 5000 werknemers,1720,Privébedrijf,Verzekeringsmaatschappijen,Verzekeringen,Minder dan 1 miljoen USD
...,...,...,...,...,...,...,...,...,...,...,...
95,Ministerie van Defensie,Den Haag,Data-analist,€ 5.450 /mnd (schatting),Salaris opgegeven door werknemer:€ 5K,10000+ medewerkers,Overheid,Aerospace en defensie,Aerospace en defensie,Onbekend/niet van toepassing,
96,Veneficus,Rotterdam,Data Analist,€ 3.850 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 5K,1 tot 50 werknemers,1998,Privébedrijf,Uitgeverijen,Media en communicatie,Onbekend/niet van toepassing
97,Breinstein Detachering,Amsterdam,Junior Data Analist,€ 2.700 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K,51 tot 200 werknemers,2010,Privébedrijf,HR-advies,HR en personeel,Onbekend/niet van toepassing
98,StarApple,Den Haag,Business Intelligence engineer,€ 4.000 /mnd (schatting),Salaris opgegeven door werknemer:€ 3K - € 5K,51 tot 200 werknemers,2008,Privébedrijf,Business consulting,Management en consulting,25-100 miljoen USD
